In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import sys
import gc
import json
import configparser
from datetime import datetime
import time

from utils.f2thresholdfinder import *
from utils.loaderjpg import *
from utils.generator import *
from utils.custommetrics import *
from utils.predictorjpg import *
from utils.file import *

from pretrained.custommodels import *

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1060 6GB (0000:01:00.0)


In [2]:
labels = ['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 
          'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 
          'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']

file_uuid = time.strftime("%Y%m%d-%H%M%S")

data_dir = 'D:/Downloads/amazon/'

rescaled_dim = 224

# model_name = 'densenet121'
# should_load_model = False
# model_filepath = None
# weights_filepath = 'D:/Downloads/amazon/bottleneck/densenet121/frozen73_20170716-171641_weights_only.h5'

model_name = 'resnet50'
should_load_model = False
model_filepath = None
weights_filepath = 'D:/Downloads/amazon/bottleneck/resnet50/frozen38_20170717-092206_weights_only.h5'

# model_name = 'vgg16'
# should_load_model = True
# model_filepath = 'D:/Downloads/amazon/bottleneck/vgg16/frozen11_20170720-065517.h5'
# weights_filepath = None

# model_name = 'vgg19'
# should_load_model = False
# model_filepath = 'D:/Downloads/amazon/bottleneck/vgg19/frozen13_20170718-183101.h5' # Bug with Keras
# weights_filepath = 'D:/Downloads/amazon/bottleneck/vgg19/frozen13_20170718-183101_weights_only.h5'


if should_load_model:
    print('loading model file: {}'.format(model_filepath))
    model = load_model(model_filepath)
else:
    print('loading model weights file: {}'.format(weights_filepath))
    model = custom_top_model(model_name, num_classes=17, num_frozen_layers=0)
    model.load_weights(weights_filepath)


loading model weights file: D:/Downloads/amazon/bottleneck/resnet50/frozen38_20170717-092206_weights_only.h5


In [3]:
df_train = pd.read_csv(data_dir + 'train_v2.csv')
x_train, y_train = load_training_set(df_train, rescaled_dim)
print(x_train.shape)
print(y_train.shape)

(40479L, 224L, 224L, 3L)
(40479L, 17L)


In [4]:
number_of_samples = x_train.shape[0]
split = int(number_of_samples * 0.90)
                     
x_valid, y_valid = x_train[split:], y_train[split:]

del x_train
del y_train

number_validations = number_of_samples - split

In [5]:
img_normalization = image_normalization_func(model_name)
valid_datagen = BottleNeckImgGenerator(normalization=img_normalization)

In [6]:
y_predictions, optimized_thresholds = predict_with_optimal_thresholds(x_valid, y_valid, valid_datagen, model)

threshold_df = pd.DataFrame({'label':labels, 
                             'optimized_threshold':optimized_thresholds})
print(threshold_df)

C:\Users\Me\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


label:0 threshold:0.29 score:0.901941455181
label:1 threshold:0.08 score:0.90539498334
label:2 threshold:0.31 score:0.905609688139
label:3 threshold:0.17 score:0.907871945561
label:4 threshold:0.04 score:0.911936839873
label:5 threshold:0.25 score:0.911977083764
label:6 threshold:0.14 score:0.914077703266
label:7 threshold:0.17 score:0.916296101521
label:8 threshold:0.21 score:0.919021713075
label:9 threshold:0.11 score:0.920021282337
label:10 threshold:0.14 score:0.92023644765
label:11 threshold:0.17 score:0.922349162987
label:12 threshold:0.15 score:0.923297033475
label:13 threshold:0.21 score:0.92330687569
label:14 threshold:0.2 score:0.927807214006
label:15 threshold:0.18 score:0.930156605232
label:16 threshold:0.12 score:0.930250803913
('>>>> Overall precision score over validation set ', 0.87324594940084077)
('>>>> Overall recall score over validation set ', 0.95798483672124968)
('>>>> Overall F2 score over validation set ', 0.93025080391310422)
                label  optimized_t

In [7]:
sample_submission_filepath = data_dir + 'sample_submission_v2.csv'
real_submission_filepath = data_dir + 'my_submissions/ttav2_submission_{}_{}.csv'.format(model_name, file_uuid)

img_normalization = image_normalization_func(model_name)
test_datagen = BottleNeckImgGenerator(normalization=img_normalization)

In [8]:
make_submission(model,
                optimized_thresholds,
                rescaled_dim, 
                labels,
                sample_submission_filepath,
                real_submission_filepath,
                test_datagen)

KeyboardInterrupt: 